In [6]:
#group data by date collected
import pandas as pd
import fastparquet
import numpy
import os
import json
from pandas.io.json import json_normalize
from json import JSONDecodeError
moredata=pd.DataFrame()
baddata=[]

def justmakeitparquetable(cell):
    if type(cell) != list:
        cell=[cell]
    return cell

def cleanup(dirtydata):
    dirtydata['A']=dirtydata['A'][0].split('\n')
    return dirtydata

def ipcleanup(row):
    iprow=[]
    for address in row['A']:
        try:
            tempstring=address.split('IN A')
            iprow.append(tempstring[1].strip())
        except IndexError:  
            pass
    row['A']=iprow
    return row    

def ttlcleanup(row):
    ttlrow=[]
    for address in row['A']:
        try:
            tempdata=address.split('IN A')
            tempdata2=tempdata[0].split(' ')
            ttlrow.append(tempdata2[1])
        except IndexError:
            pass
    row['A']=ttlrow
    return row

def exploder(index,testyboy):
    testyboy2=pd.DataFrame()
    if len(testyboy['A'])>1:
        for i in range(0,len(testyboy['A'])):
            testyseries=pd.DataFrame({'ttl':[testyboy['ttl'][i]],'benign':[testyboy['benign']],'A':[testyboy['A'][i]]})
            testyboy2=pd.concat([testyboy2,testyseries],ignore_index=True)
        return testyboy2
    return testyboy

def tostring(row):
    row['A']=row['A'][0]
    row['benign']=row['benign'][0]
    row['ttl']=row['ttl'][0]
    return row


toplevel=os.fsencode('/data/data/')
for date in os.listdir(toplevel):
    nextlevel=os.fsencode(os.fsdecode(toplevel)+os.fsdecode(date)+'/enrichment/dns/')
    datedata=pd.DataFrame()
    try:
        for file in os.listdir(nextlevel):
            try:
                with open(os.fsdecode(nextlevel)+os.fsdecode(file)) as data:
                    temp=json.load(data)                      
                    prettytemp=json_normalize(temp['data'])
                    try:
                        if len(os.fsdecode(file))<15:
                            prettytemp.insert(2,'benign',True)
                        else:
                            prettytemp.insert(2,'benign',False)
                        moredata=pd.concat([moredata,prettytemp],ignore_index=True)
                        datedata=pd.concat([datedata,prettytemp.loc[:,['benign','A']]],ignore_index=True)
                    except IndexError:
                        baddata=baddata+[os.fsdecode(nextlevel)+os.fsdecode(file)]
                    except KeyError:
                        pass
            except AttributeError:
                baddata=baddata+[os.fsdecode(nextlevel)+os.fsdecode(file)]
            except JSONDecodeError:
                baddata=baddata+[os.fsdecode(nextlevel)+os.fsdecode(file)]
        try:
            print(os.fsdecode(date)+':writing to parquet #####################################################################')
            datedata=datedata[datedata['A'].notnull()]
            
            temp=datedata.apply(cleanup,axis='columns')
            temp2=temp.apply(ipcleanup,axis='columns')
            temp3=temp.apply(ttlcleanup,axis='columns')
            temp5=temp2.copy()
            temp5.insert(1,'ttl',temp3['A'])
            
            
            temp5=temp5[temp5['A'].map(lambda x: len(x)>0)]
    
            explodedip=pd.DataFrame()
            for index,row in temp5.iterrows():
                explodedip=explodedip.append(exploder(index,row),ignore_index=True)
            explodedip=explodedip.applymap(justmakeitparquetable)
            explodedip=explodedip.apply(tostring,axis='columns')
            explodedip=explodedip.drop_duplicates(['A','Benign']).reset_index(drop=True)
            
            new=explodedip['A'].str.split(".",n=-1,expand=True)
            explodedip['1st Octet']=new[0]
            explodedip['2nd Octet']=new[1]
            explodedip['3rd Octet']=new[2]
            explodedip['4th Octet']=new[3]
            
            explodedip.drop(columns=['A'],inplace=True)
            explodedip.to_parquet('/stuff/data collection and cleanup/dns data by date/'+os.fsdecode(date))
        except KeyError:
            pass
    except FileNotFoundError:
        baddata=baddata+[os.fsdecode(date)]       

/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:76: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
/home/ubuntu/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:1367: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


2019-02-23:writing to parquet #####################################################################
2019-02-24:writing to parquet #####################################################################
2019-02-25:writing to parquet #####################################################################
2019-02-26:writing to parquet #####################################################################
2019-02-27:writing to parquet #####################################################################
2019-02-28:writing to parquet #####################################################################
2019-03-01:writing to parquet #####################################################################
2019-03-02:writing to parquet #####################################################################
2019-03-03:writing to parquet #####################################################################
2019-03-04:writing to parquet #####################################################################


2019-05-19:writing to parquet #####################################################################
2019-05-20:writing to parquet #####################################################################
2019-05-21:writing to parquet #####################################################################
2019-05-22:writing to parquet #####################################################################
2019-05-23:writing to parquet #####################################################################
2019-05-24:writing to parquet #####################################################################
2019-05-25:writing to parquet #####################################################################
2019-05-26:writing to parquet #####################################################################
2019-05-27:writing to parquet #####################################################################
2019-05-28:writing to parquet #####################################################################


In [2]:
temp=pd.read_parquet('/stuff/data collection and cleanup/dns data by date/2019-02-23')
compare=pd.read_parquet('/stuff/dnsdata')

In [4]:
temp

,benign,ttl,1st Octet,2nd Octet,3rd Octet,4th Octet
0,0.0,299,64,6,231,77
1,0.0,299,43,245,53,42
2,0.0,14399,159,69,121,73
3,0.0,299,203,157,65,4
4,0.0,3599,119,47,118,96
5,0.0,3599,172,245,13,172
6,0.0,3600,49,50,249,70
7,0.0,21599,203,96,152,15
8,0.0,14399,192,99,151,130
9,0.0,2904,185,33,54,7


In [5]:
compare

,indicator,benign,A,AAAA,CAA,CNAME,DNAME,LOC,MX,RP,SIG,SSHFP,success
1,[baptist411.com],False,[baptist411.com. 299 IN A 64.6.231.77],[No Answer],[No Answer],[No Answer],[No Answer],[No Answer],[baptist411.com. 299 IN MX 0 baptist411.com.],[No Answer],[No Answer],[No Answer],True
2,[fantalebooks.com],False,[fantalebooks.com. 299 IN A 43.245.53.42],[No Answer],[No Answer],[No Answer],[No Answer],[No Answer],[fantalebooks.com. 299 IN MX 10 mta.hosts.net....,[No Answer],[No Answer],[No Answer],True
3,[nakulsoft.in],False,[nakulsoft.in. 14399 IN A 159.69.121.73],[No Answer],[No Answer],[No Answer],[No Answer],[No Answer],[nakulsoft.in. 14399 IN MX 0 nakulsoft.in.],[No Answer],[No Answer],[No Answer],True
4,[yimsodsai.com],False,[yimsodsai.com. 299 IN A 203.157.65.4],[No Answer],[No Answer],[No Answer],[No Answer],[No Answer],[No Answer],[No Answer],[No Answer],[No Answer],True
7,[eden.it-guys.net.nz],False,[eden.it-guys.net.nz. 3599 IN A 119.47.118.96],[No Answer],[No Answer],[No Answer],[No Answer],[No Answer],[eden.it-guys.net.nz. 3599 IN MX 10 mail.eden....,[No Answer],[No Answer],[No Answer],True
8,[fantalebooks.com],False,[fantalebooks.com. 299 IN A 43.245.53.42],[No Answer],[No Answer],[No Answer],[No Answer],[No Answer],[fantalebooks.com. 299 IN MX 10 mta.hosts.net....,[No Answer],[No Answer],[No Answer],True
9,[yimsodsai.com],False,[yimsodsai.com. 299 IN A 203.157.65.4],[No Answer],[No Answer],[No Answer],[No Answer],[No Answer],[No Answer],[No Answer],[No Answer],[No Answer],True
11,[baptist411.com],False,[baptist411.com. 299 IN A 64.6.231.77],[No Answer],[No Answer],[No Answer],[No Answer],[No Answer],[baptist411.com. 299 IN MX 0 baptist411.com.],[No Answer],[No Answer],[No Answer],True
12,[nakulsoft.in],False,[nakulsoft.in. 14399 IN A 159.69.121.73],[No Answer],[No Answer],[No Answer],[No Answer],[No Answer],[nakulsoft.in. 14399 IN MX 0 nakulsoft.in.],[No Answer],[No Answer],[No Answer],True
13,[eden.it-guys.net.nz],False,[eden.it-guys.net.nz. 3599 IN A 119.47.118.96],[No Answer],[No Answer],[No Answer],[No Answer],[No Answer],[eden.it-guys.net.nz. 3547 IN MX 10 mail.eden....,[No Answer],[No Answer],[No Answer],True
